In [23]:
import numpy as np
import math
import copy
import datetime
import pandas as pd
import json
import matplotlib.pyplot as plt

### Description of add_up_a_decomposition
The function takes as inputs:
1. a decomposition (x_list, BN_matrices_array) output by SER 1, SER 2, GER (x_list is a list, whereas BN_matrices_array is a 2D np array).
2. The dimension of the output TPM row_dim, col_dim

The function then sums up (x_list, BN_matrices_array) and outputs the resulting TPM (row_dim-by-col_dim).

In [4]:
def add_up_a_decomposition(x_list, BN_matrices_array, row_dim, col_dim):
    output_TPM = np.zeros((row_dim, col_dim), dtype=int)
    num_BN_matrices = len(x_list)
    
    for k in range(num_BN_matrices):
        for col in range(col_dim):
            output_TPM[BN_matrices_array[k, col], col] += x_list[k]
    
    return output_TPM

# Implementation of the arXiv Version of GER

## Order of execution: get_col_indices, first_occurrence, column_freq, find_v_list, GERESA, f_score, new_GER.

This function requires two inputs:
1. $P$ is a $2^{n} \times 2^{n}$ numpy array.
2. $z$ is a real number greater than 1.

This function outputs:
1. The length of the decomposition $K$.
2. A list of positive real numbers x_list.
3. A 2D numpy array representing a list of BN matrices A_array.

In [11]:
def new_GER(P, z):
    row_num, col_num = P.shape
    zero_matrix = np.zeros((row_num, col_num), dtype=int)
    
    R = copy.deepcopy(P)
    K = 0
    x_list = []
    BN_list = []
    
    while (not np.array_equal(R, zero_matrix)):
        K += 1
        B = np.min(np.max(R, axis=0))
        v_list = find_v_list(R, B)
        x = 0
        score = -math.inf
        
        for v in v_list:
            temp_A = GERESA(R, v) # temp_A should be a list of length col_num representing a BN matrix.
            
            R_subtract_v_temp_A = copy.deepcopy(R)
            for j in range(col_num):
                R_subtract_v_temp_A[temp_A[j], j] -= v
            
            temp_score = f_score(R_subtract_v_temp_A, z)
            
            if ((temp_score > score) or ((temp_score == score) and (v > x))):
                score = temp_score
                x = v
                A = temp_A
        
        for j in range(col_num):
            R[A[j], j] -= x
        
        x_list.append(x)
        BN_list.append(A)
        
    return K, x_list, np.array(BN_list)

This function requires two inputs:
1. an integer $v$.
2. A 2D numpy array an_array consisting of integers.

The function outputs the column frequency of $v$ in an_array.

In [7]:
def column_freq(v, an_array):
    row_num, col_num = an_array.shape
    col_freq = 0
    
    for j in range(col_num):
        if v in an_array[:, j]:
            col_freq += 1
    
    return col_freq

This function outputs the list of positive entries of $R$ which are not greater than $B$ (upper_bound) and attain the maximum column frequency in $R$.

In [8]:
def find_v_list(R, upper_bound):
    row_num, col_num = R.shape
    classify_entries = [set() for i in range(col_num + 1)]
    
    for i in range(row_num):
        for j in range(col_num):
            if (R[i, j] > 0 and R[i, j] <= upper_bound):
                classify_entries[column_freq(R[i, j], R)].add(R[i, j])
    
    empty_set = set()
    
    for freq in range(col_num, 0, -1):
        if classify_entries[freq] != empty_set:
            return list(classify_entries[freq])

In [9]:
def GERESA(R, v):
    row_num, col_num = R.shape
    A = [0] * col_num
    R_copy = copy.deepcopy(R)
    selected_columns = []
    col_indices_v_R = get_col_indices(v, R)
    col_indices_v_R_complement = set(range(col_num)) - col_indices_v_R
    
    for j in col_indices_v_R:
        A[j] = first_occurrence(v, R[:, j])
        R_copy[A[j], j] = 0
        selected_columns.append(j)
    
    for j in col_indices_v_R_complement:
        # Compute a dictionary with keys in Larger(v, R(:, j)) and values being associated column frequencies.
        dict_KLarger_VColFreq = dict()
        for i in range(row_num):
            if R[i, j] > v:
                dict_KLarger_VColFreq[i] = column_freq(R[i, j] - v, R_copy[:, selected_columns])
        
        # Set A[j] to the right choice from range(row_num).
        A[j] = max(dict_KLarger_VColFreq.keys(), key=dict_KLarger_VColFreq.get)
        
        R_copy[A[j], j] = R[A[j], j] - v
        selected_columns.append(j)
        
    return A

This function requires two inputs:
1. an integer $v$.
2. A 2D numpy array an_array consisting of integers.

The function outputs the set $\text{Col_indices}(v, \text{an_array})$.

In [5]:
def get_col_indices(v, an_array):
    row_num, col_num = an_array.shape
    col_indices = []
    
    for j in range(col_num):
        if v in an_array[:, j]:
            col_indices.append(j)
    
    return set(col_indices)

This function requires two inputs:
1. An integer $v$.
2. A 1D numpy array array_1D such that v is an element of array_1D.

The function outputs the index of the first occurrence of $v$ in array_1D.

In [6]:
def first_occurrence(v, array_1D):
    length = array_1D.size
    
    for i in range(length):
        if v == array_1D[i]:
            return i
    
    return -1

This function requires two inputs:
1. An integer 2D array array_2D.
2. A positive real number $z > 1$.

The function outputs a score.

In [10]:
def f_score(array_2D, z):
    row_num, col_num = array_2D.shape
    score = 0
    checked_positive_entries = set()
    
    for i in range(row_num):
        for j in range(col_num):
            if (array_2D[i, j] > 0) and (not array_2D[i, j] in checked_positive_entries):
                score += z ** column_freq(array_2D[i, j], array_2D)
                checked_positive_entries.add(array_2D[i, j])
    
    return score

# Parameter Sensitivity Analysis ($8 \times 8$ 5000-TPMs)

In [24]:
# Custom JSON encoder that handles NumPy types (created by GPT-o3 on poe.com)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        elif isinstance(obj, (np.floating,)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

### What the following code does:

For each matrix:
1. Execute GER on it with $z=2, 3, \ldots, 100$. 
2. Save the tuple of the output decompositions.
3. Plot a graph of the lengths of decompositions against z. Save the graph.

In [26]:
for k in range(10):
    lengths_of_decoms = []
    filepath = './random 5000-TPMs/dim8_matrix_' + str(k) + '.npy'
    pTPM = np.load(filepath)
    
    for z in range(2, 101):
        decomposition = new_GER(pTPM, z)
        lengths_of_decoms.append(decomposition[0])
        decom_path = './output decompositions (5000-TPMs)/dim8_matrix_' + str(k) + '/z_' + str(z) + '.json'
        
        with open(decom_path, "w") as out:
            json.dump(decomposition, out, cls=NumpyEncoder)
    
    plt.figure()
    z_axis = list(range(2, 101))
    plt.plot(z_axis, lengths_of_decoms, color='r')
    plt.xlabel("values of z")
    plt.ylabel("lengths of decompositions")
    
    figure_path = "./line plots (5000-TPMs)/dim8_matrix_" + str(k) + "_plot.png"
    plt.savefig(figure_path)
    plt.close()

### What the following code does:

For each matrix:
1. Execute GER on it with $z = 1.1, 1.2, \ldots, 29.9, 30$.
2. Save the tuple of the output decompositions.
3. Plot a graph of the lengths of decompositions against z. Save the graph.

In [41]:
for k in range(10):
    print('Start of k =', k, '    ', datetime.datetime.now())
    lengths_of_decoms = []
    filepath = './random 5000-TPMs/dim8_matrix_' + str(k) + '.npy'
    pTPM = np.load(filepath)
    
    for z_int_part in range(1, 30):
        for z_frac_part in range(1, 11):
            decomposition = new_GER(pTPM, z_int_part + 0.1 * z_frac_part)
            lengths_of_decoms.append(decomposition[0])
            decom_path = './output decompositions (5000-TPMs)/dim8_matrix_' + str(k) + '/fz_' + \
                         str(z_int_part) + 'dot' + str(z_frac_part) + '.json'
        
            with open(decom_path, "w") as out:
                json.dump(decomposition, out, cls=NumpyEncoder)
            
            if (z_frac_part == 10):
                print("finished z =", z_int_part + 0.1 * z_frac_part)
            
    plt.figure()
    z_axis = np.linspace(1.1, 30, 290)
    plt.plot(z_axis, lengths_of_decoms, color='r')
    plt.xlabel("values of z")
    plt.ylabel("lengths of decompositions")
    
    figure_path = "./line plots (5000-TPMs)/dim8_matrix_" + str(k) + "_plot_float_z.png"
    plt.savefig(figure_path)
    plt.close()
    
    print("End of k =", k)
    print()

Start of k = 0      2025-04-07 03:58:08.842451
finished z = 2.0
finished z = 3.0
finished z = 4.0
finished z = 5.0
finished z = 6.0
finished z = 7.0
finished z = 8.0
finished z = 9.0
finished z = 10.0
finished z = 11.0
finished z = 12.0
finished z = 13.0
finished z = 14.0
finished z = 15.0
finished z = 16.0
finished z = 17.0
finished z = 18.0
finished z = 19.0
finished z = 20.0
finished z = 21.0
finished z = 22.0
finished z = 23.0
finished z = 24.0
finished z = 25.0
finished z = 26.0
finished z = 27.0
finished z = 28.0
finished z = 29.0
finished z = 30.0
End of k = 0

Start of k = 1      2025-04-07 04:00:32.697841
finished z = 2.0
finished z = 3.0
finished z = 4.0
finished z = 5.0
finished z = 6.0
finished z = 7.0
finished z = 8.0
finished z = 9.0
finished z = 10.0
finished z = 11.0
finished z = 12.0
finished z = 13.0
finished z = 14.0
finished z = 15.0
finished z = 16.0
finished z = 17.0
finished z = 18.0
finished z = 19.0
finished z = 20.0
finished z = 21.0
finished z = 22.0
finished

# Parameter Sensitivity Analysis ($16 \times 16$ 5000-TPMs)

### What the following code does:

For each matrix:
1. Execute GER on it with $z=2, 3, \ldots, 100$. 
2. Save the tuple of the output decompositions.
3. Plot a graph of the lengths of decompositions against z. Save the graph.

In [32]:
for k in range(10):
    print('Start of k =', k, '    ', datetime.datetime.now())
    lengths_of_decoms = []
    filepath = './random 5000-TPMs/dim16_matrix_' + str(k) + '.npy'
    pTPM = np.load(filepath)
    
    for z in range(2, 101):
        decomposition = new_GER(pTPM, z)
        lengths_of_decoms.append(decomposition[0])
        decom_path = './output decompositions (5000-TPMs)/dim16_matrix_' + str(k) + '/z_' + str(z) + '.json'
        
        with open(decom_path, "w") as out:
            json.dump(decomposition, out, cls=NumpyEncoder)
        
        if z in [25, 50, 75, 100]:
            print("finished z =", z)
        
    plt.figure()
    z_axis = list(range(2, 101))
    plt.plot(z_axis, lengths_of_decoms, color='r')
    plt.xlabel("values of z")
    plt.ylabel("lengths of decompositions")
    
    figure_path = "./line plots (5000-TPMs)/dim16_matrix_" + str(k) + "_plot.png"
    plt.savefig(figure_path)
    plt.close()
    print("End of k =", k)
    print()

Start of k = 0      2025-04-07 00:09:59.156986
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 0

Start of k = 1      2025-04-07 00:12:13.215192
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 1

Start of k = 2      2025-04-07 00:14:10.222065
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 2

Start of k = 3      2025-04-07 00:17:10.982043
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 3

Start of k = 4      2025-04-07 00:20:04.449800
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 4

Start of k = 5      2025-04-07 00:22:18.012218
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 5

Start of k = 6      2025-04-07 00:25:03.587756
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 6

Start of k = 7      2025-04-07 00:27:34.695984
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of

### What the following code does:

For each matrix:
1. Execute GER on it with $z = 1.1, 1.2, \ldots, 29.9, 30$.
2. Save the tuple of the output decompositions.
3. Plot a graph of the lengths of decompositions against z. Save the graph.

In [44]:
for k in range(10):
    print('Start of k =', k, '    ', datetime.datetime.now())
    lengths_of_decoms = []
    filepath = './random 5000-TPMs/dim16_matrix_' + str(k) + '.npy'
    pTPM = np.load(filepath)
    
    for z_int_part in range(1, 30):
        for z_frac_part in range(1, 11):
            decomposition = new_GER(pTPM, z_int_part + 0.1 * z_frac_part)
            lengths_of_decoms.append(decomposition[0])
            decom_path = './output decompositions (5000-TPMs)/dim16_matrix_' + str(k) + '/fz_' +\
                         str(z_int_part) + 'dot' + str(z_frac_part) + '.json'
        
            with open(decom_path, "w") as out:
                json.dump(decomposition, out, cls=NumpyEncoder)
            
            if (z_frac_part == 10):
                print("finished z =", z_int_part + 0.1 * z_frac_part)
            
    plt.figure()
    z_axis = np.linspace(1.1, 30, 290)
    plt.plot(z_axis, lengths_of_decoms, color='r')
    plt.xlabel("values of z")
    plt.ylabel("lengths of decompositions")
    
    figure_path = "./line plots (5000-TPMs)/dim16_matrix_" + str(k) + "_plot_float_z.png"
    plt.savefig(figure_path)
    plt.close()
    
    print("End of k =", k)
    print()

Start of k = 0      2025-04-07 14:44:48.473650
finished z = 2.0
finished z = 3.0
finished z = 4.0
finished z = 5.0
finished z = 6.0
finished z = 7.0
finished z = 8.0
finished z = 9.0
finished z = 10.0
finished z = 11.0
finished z = 12.0
finished z = 13.0
finished z = 14.0
finished z = 15.0
finished z = 16.0
finished z = 17.0
finished z = 18.0
finished z = 19.0
finished z = 20.0
finished z = 21.0
finished z = 22.0
finished z = 23.0
finished z = 24.0
finished z = 25.0
finished z = 26.0
finished z = 27.0
finished z = 28.0
finished z = 29.0
finished z = 30.0
End of k = 0

Start of k = 1      2025-04-07 14:52:32.535538
finished z = 2.0
finished z = 3.0
finished z = 4.0
finished z = 5.0
finished z = 6.0
finished z = 7.0
finished z = 8.0
finished z = 9.0
finished z = 10.0
finished z = 11.0
finished z = 12.0
finished z = 13.0
finished z = 14.0
finished z = 15.0
finished z = 16.0
finished z = 17.0
finished z = 18.0
finished z = 19.0
finished z = 20.0
finished z = 21.0
finished z = 22.0
finished

# Parameter Sensitivity Analysis ($32 \times 32$ 5000-TPMs)

### What the following code does:

For each matrix:
1. Execute GER on it with $z=2, 3, \ldots, 100$. 
2. Save the tuple of the output decompositions.
3. Plot a graph of the lengths of decompositions against z. Save the graph.

In [33]:
for k in range(10):
    print('Start of k =', k, '    ', datetime.datetime.now())
    lengths_of_decoms = []
    filepath = './random 5000-TPMs/dim32_matrix_' + str(k) + '.npy'
    pTPM = np.load(filepath)
    
    for z in range(2, 101):
        decomposition = new_GER(pTPM, z)
        lengths_of_decoms.append(decomposition[0])
        decom_path = './output decompositions (5000-TPMs)/dim32_matrix_' + str(k) + '/z_' + str(z) + '.json'
        
        with open(decom_path, "w") as out:
            json.dump(decomposition, out, cls=NumpyEncoder)
        
        if z in [25, 50, 75, 100]:
            print("finished z =", z)
        
    plt.figure()
    z_axis = list(range(2, 101))
    plt.plot(z_axis, lengths_of_decoms, color='r')
    plt.xlabel("values of z")
    plt.ylabel("lengths of decompositions")
    
    figure_path = "./line plots (5000-TPMs)/dim32_matrix_" + str(k) + "_plot.png"
    plt.savefig(figure_path)
    plt.close()
    print("End of k =", k)
    print()

Start of k = 0      2025-04-07 00:44:42.947937
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 0

Start of k = 1      2025-04-07 01:01:01.880590
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 1

Start of k = 2      2025-04-07 01:18:19.321774
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 2

Start of k = 3      2025-04-07 01:35:52.855414
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 3

Start of k = 4      2025-04-07 01:54:00.400795
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 4

Start of k = 5      2025-04-07 02:11:28.265009
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 5

Start of k = 6      2025-04-07 02:28:01.318860
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of k = 6

Start of k = 7      2025-04-07 02:52:17.896049
finished z = 25
finished z = 50
finished z = 75
finished z = 100
End of

### What the following code does:

For each matrix:
1. Execute GER on it with $z = 1.1, 1.2, \ldots, 29.9, 30$.
2. Save the tuple of the output decompositions.
3. Plot a graph of the lengths of decompositions against z. Save the graph.

In [43]:
for k in range(10):
    print('Start of k =', k, '    ', datetime.datetime.now())
    lengths_of_decoms = []
    filepath = './random 5000-TPMs/dim32_matrix_' + str(k) + '.npy'
    pTPM = np.load(filepath)
    
    for z_int_part in range(1, 30):
        for z_frac_part in range(1, 11):
            decomposition = new_GER(pTPM, z_int_part + 0.1 * z_frac_part)
            lengths_of_decoms.append(decomposition[0])
            decom_path = './output decompositions (5000-TPMs)/dim32_matrix_' + str(k) + '/fz_' +\
                         str(z_int_part) + 'dot' + str(z_frac_part) + '.json'
        
            with open(decom_path, "w") as out:
                json.dump(decomposition, out, cls=NumpyEncoder)
            
            if (z_frac_part == 10):
                print("finished z =", z_int_part + 0.1 * z_frac_part)
            
    plt.figure()
    z_axis = np.linspace(1.1, 30, 290)
    plt.plot(z_axis, lengths_of_decoms, color='r')
    plt.xlabel("values of z")
    plt.ylabel("lengths of decompositions")
    
    figure_path = "./line plots (5000-TPMs)/dim32_matrix_" + str(k) + "_plot_float_z.png"
    plt.savefig(figure_path)
    plt.close()
    
    print("End of k =", k)
    print()

Start of k = 0      2025-04-07 04:21:21.504555
finished z = 2.0
finished z = 3.0
finished z = 4.0
finished z = 5.0
finished z = 6.0
finished z = 7.0
finished z = 8.0
finished z = 9.0
finished z = 10.0
finished z = 11.0
finished z = 12.0
finished z = 13.0
finished z = 14.0
finished z = 15.0
finished z = 16.0
finished z = 17.0
finished z = 18.0
finished z = 19.0
finished z = 20.0
finished z = 21.0
finished z = 22.0
finished z = 23.0
finished z = 24.0
finished z = 25.0
finished z = 26.0
finished z = 27.0
finished z = 28.0
finished z = 29.0
finished z = 30.0
End of k = 0

Start of k = 1      2025-04-07 05:01:25.954041
finished z = 2.0
finished z = 3.0
finished z = 4.0
finished z = 5.0
finished z = 6.0
finished z = 7.0
finished z = 8.0
finished z = 9.0
finished z = 10.0
finished z = 11.0
finished z = 12.0
finished z = 13.0
finished z = 14.0
finished z = 15.0
finished z = 16.0
finished z = 17.0
finished z = 18.0
finished z = 19.0
finished z = 20.0
finished z = 21.0
finished z = 22.0
finished